# Calculate housing emissions
In this notebook, I will calculate operational carbon emissions (from electricity and gas use) associated with housing in the Netherlands, from 2011 until now. This requires the following steps: 
- Download all cbs buurten data from 2011 until now 
- Transfer data to pgsql database 
- Calculate operational emissions using emission density 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

# Download data from 2011 until now
**Where can the data be downloaded?**
- [This webpage](https://www.cbs.nl/nl-nl/reeksen/publicatie/kerncijfers-wijken-en-buurten) of CBS seems to have the key figures of neighborhoods from 2004 - 2023. 
- The files can be downloaded as excel files, which don't have geometry. But has the neighborhood name and id, which can be used to get geometries from nl_buurten. 

**What columns are needed for calculating operational emissions?** 

Emissions are calculated per neighborhood (buurt) using the following formula:

`operational emissions` = `n households` * (`av electricity use per household (kWh)` * `emissions intensity for electricity (tCO2eq/kWh)` + `av gas use per household (m3)` * `emissions intensity for gas (tCO2eq/m3)`)

So the columns needed from cbs data are: 
- `AANTAL_HH` - n households
- `P_ELEK_TOT` - average electricity use per household
- `P_GAS_TOT` - average gas use per household 

In [5]:
df_file = pd.read_excel('data/cbs/kwb-2011.xls')

In [13]:
df = df_file.copy()
df = df[df.RECS == 'Buurt']

for column in ['AANTAL_HH', 'P_ELEK_TOT', 'P_GAS_TOT']: 
    if column in df.columns: 
        print(f'{column} is in the excel')
    else: 
        print(f'{column} is not in the excel')

AANTAL_HH is in the excel
P_ELEK_TOT is in the excel
P_GAS_TOT is in the excel


In [ ]:
gpkg 

In [16]:
df

,CODE_10POS11,GWB_CODE11,GWB_NAAM11_60pos,GWB_NAAM11_40pos,RECS,GM_CODE,WK_CODE,BU_CODE,POSTCODE,DEK_PERC,...,M-N Zakelijke dienstverlening,"R-U Cultuur, recreatie overige dienstverlening",GEBOO_TOT,P_GEBOO,STERFT_TOT,P_STERFT,AO_UIT_TOT,P_AO_UIT,WW_UIT_TOT,P_WW_UIT
3,30000,30000,Appingedam-Centrum,Appingedam-Centrum,Buurt,3.0,0.0,0.0,9901,1,...,20,20,10,4,75,32,190,131,50,36
4,30001,30001,Appingedam-West,Appingedam-West,Buurt,3.0,0.0,1.0,9903,5,...,20,20,15,4,15,5,140,68,80,37
5,30002,30002,Appingedam-Oost,Appingedam-Oost,Buurt,3.0,0.0,2.0,9902,1,...,20,20,60,9,60,10,390,103,160,41
6,30007,30007,Verspreide huizen Damsterdiep en Eemskanaal,Verspr. huizen Damsterdiep en Eemskanaal,Buurt,3.0,0.0,7.0,9903,4,...,0,0,5,10,0,X,20,93,0,10
7,30008,30008,Verspreide huizen ten zuiden van Eemskanaal,Verspr. huizen ten zuiden van Eemskanaal,Buurt,3.0,0.0,8.0,9902,1,...,5,0,0,0,0,0,x,x,x,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14767,1019870300,19870300,Muntendam met Oude Verlaat,Muntendam met Oude Verlaat,Buurt,1987.0,3.0,0.0,9649,1,...,25,30,50,10,25,6,370,114,110,33
14768,19870302,19870302,Tussenklappen,Tussenklappen,Buurt,1987.0,3.0,2.0,9649,1,...,x,x,0,X,0,0,x,x,x,x
14769,19870303,19870303,Tripscompagnie,Tripscompagnie,Buurt,1987.0,3.0,3.0,9633,1,...,x,x,0,0,0,0,10,108,0,22
14770,19870304,19870304,Borgercompagnie (gedeeltelijk),Borgercompagnie (gedeeltelijk),Buurt,1987.0,3.0,4.0,9632,1,...,x,x,0,0,0,0,x,x,x,x
